In [3]:
import csv
import re

# Load movies from movies_genres.csv
with open('movie_genres.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    data = list(reader)

# Remove the year from the title
for row in data:
    row[1] = re.sub(r'\s*\(\d{4}\)\s*', '', row[1])

# Write the modified data back to the CSV file
with open('movies_genres.csv', 'w', newline='', encoding= 'utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(data)



In [4]:
import csv
import json

import rdflib

# Paths to the files
WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')

# Read the relations and their IDs
with open('embeds/relation_ids.del', 'r', encoding='utf-8') as ifile:
    rel2id = {rdflib.term.URIRef(rel): int(idx) for idx, rel in csv.reader(ifile, delimiter='\t')}

# Read the labels and their corresponding relations
with open('property.json', 'r', encoding='utf-8') as ifile:
    rel2lbl = json.load(ifile)

# Map the relations to their label
# 
# new_rel2lbl = {str(rel)[len(WDT):]: rel2lbl[str(rel)[len(WDT):]] for rel in rel2id.keys() if str(rel)[len(WDT):] in rel2lbl}
new_rel2lbl = {str(rel)[len(WDT):]: rel2lbl.get(str(rel)[len(WDT):], None) for rel in rel2id.keys()}


# Write the result to the new JSON file
with open('relations.json', 'w', encoding='utf-8') as ofile:
    json.dump(new_rel2lbl, ofile, ensure_ascii=False, indent=4)


In [2]:
import json

# Load the data from your JSON files
with open('my_property.json', 'r', encoding='utf-8') as f:
    my_property = json.load(f)

with open('rel2lbl.json', 'r', encoding='utf-8') as f:
    rel2lbl = json.load(f)

# Check for items in rel2lbll if not in my_property value[0]
for key, value in rel2lbl.items():
    if key not in my_property or value not in my_property[key]:
        # If not present, add it to my_property
        if key in my_property:
            my_property[key].append(value)
        else:
            my_property[key] = [value]

# Save the updated my_property back to the file
with open('my_property.json', 'w') as f:
    json.dump(my_property, f, indent=4)


In [3]:
import json
from SPARQLWrapper import SPARQLWrapper, JSON

# Load the data from your JSON file
with open('my_property.json', 'r') as f:
    my_property = json.load(f)

# Initialize the SPARQL endpoint
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

# Iterate over the items in my_property
for key, values in my_property.items():
    # If the first value is null
    if values[0] is None:
        # Construct the SPARQL query
        query = f"""
        SELECT ?item ?itemLabel WHERE {{
            wd:{key} rdfs:label ?itemLabel .
            FILTER(LANG(?itemLabel) = "en") .
        }}
        """
        # Set the query and the return format
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)

        # Execute the query and get the results
        results = sparql.query().convert()

        # If there are results
        if results["results"]["bindings"]:
            # Get the label of the first result
            label = results["results"]["bindings"][0]["itemLabel"]["value"]

            # Replace the null value with the label
            my_property[key][0] = label

# Save the updated my_property back to the file
with open('my_property.json', 'w') as f:
    json.dump(my_property, f, indent=4)

        